In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)

from sklearn import *
from lightgbm import LGBMRegressor

/Users/amane/.pyenv/versions/3.6.1/envs/compe/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/amane/.pyenv/versions/3.6.1/envs/compe/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/amane/.pyenv/versions/3.6.1/envs/compe/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into w

In [2]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor

data = {
    'tra': pd.read_csv('../input/air_visit_data.csv'),
    'as': pd.read_csv('../input/air_store_info.csv'),
    'hs': pd.read_csv('../input/hpg_store_info.csv'),
    'ar': pd.read_csv('../input/air_reserve.csv'),
    'hr': pd.read_csv('../input/hpg_reserve.csv'),
    'id': pd.read_csv('../input/store_id_relation.csv'),
    'tes': pd.read_csv('../input/sample_submission.csv'),
    'hol': pd.read_csv('../input/date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }

data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    tmp1 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

#sure it can be compressed...
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 
# NEW FEATURES FROM Georgii Vyshnia
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))
lbl = preprocessing.LabelEncoder()
# for i in range(10):
#     stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
#     stores['air_area_name'+str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
# stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
# stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date']) 

train = pd.merge(train, stores, how='left', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])

for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2

# NEW FEATURES FROM JMBULL
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

# NEW FEATURES FROM Georgii Vyshnia
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

# lbl = preprocessing.LabelEncoder()
# train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
# test['air_store_id2'] = lbl.transform(test['air_store_id'])

train = train.fillna(-1)
test = test.fillna(-1)

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [5]:
def target_encode(col):
    global train, test
    ref = pd.DataFrame({
        col+'_target_mean': train.loc[~val_id].groupby(col)['visitors'].mean(),
        col+'_target_std': train.loc[~val_id].groupby(col)['visitors'].std()
    })
    train = pd.merge(left=train, right=ref, left_on=col, right_index=True)
    ref = train.groupby(col)['visitors'].mean().to_frame(name=col+'_target')
    ref = pd.DataFrame({
        col+'_target_mean': train.groupby(col)['visitors'].mean(),
        col+'_target_std': train.groupby(col)['visitors'].std()
    })
    test = pd.merge(left=test, right=ref, left_on=col, right_index=True)

In [9]:
val_id = ((train.year == 2017) & (train.month >= 4))
print('val ratio:', f'{len(train.loc[val_id])/len(train):.2%}')

val ratio: 6.26%


In [10]:
target_encode('air_store_id')
target_encode('air_genre_name')
target_encode('air_area_name')

In [11]:
col = [c for c in train if c not in ['id', 'air_store_id', 'air_area_name', 'air_genre_name', 'visit_date','visitors']]
X_trn = train.loc[~val_id, col]
y_trn = np.log1p(train.loc[~val_id, 'visitors'].values)
X_val = train.loc[val_id, col]
y_val = np.log1p(train.loc[val_id, 'visitors'].values)
X_all = train[col]
y_all = np.log1p(train['visitors'].values)
X_tst = test[col]

In [78]:
from sklearn.model_selection import RandomizedSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
import scipy.stats

In [75]:
cv = PredefinedSplit((val_id-1).values)

In [91]:
param_dist = {
    'random_state': [77],
    'num_leaves': [40, 50, 60, 70, 80, 90, 100],
    'n_estimators': [200, 300, 400, 500, 600, 700, 800],
    'learning_rate': scipy.stats.uniform(0.03, 0.2),
    'min_child_samples': scipy.stats.randint(10, 50),
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    'subsample': [0.7, 0.8, 0.9, 1.0],
}
rscv = RandomizedSearchCV(LGBMRegressor(), param_dist, n_iter=100, 
                          scoring=make_scorer(RMSLE,greater_is_better=False),
                         cv=cv, verbose=2)

In [92]:
rscv.fit(X_all, y_all)

Fitting 1 folds for each of 100 candidates, totalling 100 fits
[CV] colsample_bytree=0.8, learning_rate=0.16211226507031734, min_child_samples=10, n_estimators=500, num_leaves=70, random_state=77, subsample=0.8 
[CV]  colsample_bytree=0.8, learning_rate=0.16211226507031734, min_child_samples=10, n_estimators=500, num_leaves=70, random_state=77, subsample=0.8, total=  19.2s
[CV] colsample_bytree=0.9, learning_rate=0.09495189146724171, min_child_samples=40, n_estimators=600, num_leaves=80, random_state=77, subsample=0.9 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   24.8s remaining:    0.0s


[CV]  colsample_bytree=0.9, learning_rate=0.09495189146724171, min_child_samples=40, n_estimators=600, num_leaves=80, random_state=77, subsample=0.9, total=  28.1s
[CV] colsample_bytree=0.7, learning_rate=0.04158026190567394, min_child_samples=39, n_estimators=600, num_leaves=90, random_state=77, subsample=1.0 
[CV]  colsample_bytree=0.7, learning_rate=0.04158026190567394, min_child_samples=39, n_estimators=600, num_leaves=90, random_state=77, subsample=1.0, total=  28.6s
[CV] colsample_bytree=0.9, learning_rate=0.18697924442721267, min_child_samples=30, n_estimators=800, num_leaves=90, random_state=77, subsample=0.7 
[CV]  colsample_bytree=0.9, learning_rate=0.18697924442721267, min_child_samples=30, n_estimators=800, num_leaves=90, random_state=77, subsample=0.7, total=  34.3s
[CV] colsample_bytree=1.0, learning_rate=0.12290353172646382, min_child_samples=25, n_estimators=800, num_leaves=60, random_state=77, subsample=0.7 
[CV]  colsample_bytree=1.0, learning_rate=0.12290353172646382

[CV]  colsample_bytree=1.0, learning_rate=0.11948570068073813, min_child_samples=46, n_estimators=500, num_leaves=90, random_state=77, subsample=0.7, total=  22.9s
[CV] colsample_bytree=0.9, learning_rate=0.1585142367365513, min_child_samples=19, n_estimators=200, num_leaves=100, random_state=77, subsample=1.0 
[CV]  colsample_bytree=0.9, learning_rate=0.1585142367365513, min_child_samples=19, n_estimators=200, num_leaves=100, random_state=77, subsample=1.0, total=  10.2s
[CV] colsample_bytree=0.8, learning_rate=0.14279659681228046, min_child_samples=36, n_estimators=400, num_leaves=100, random_state=77, subsample=0.7 
[CV]  colsample_bytree=0.8, learning_rate=0.14279659681228046, min_child_samples=36, n_estimators=400, num_leaves=100, random_state=77, subsample=0.7, total=  19.6s
[CV] colsample_bytree=0.8, learning_rate=0.08884143039897, min_child_samples=21, n_estimators=400, num_leaves=70, random_state=77, subsample=0.8 
[CV]  colsample_bytree=0.8, learning_rate=0.08884143039897, mi

[CV]  colsample_bytree=0.7, learning_rate=0.06123980118766654, min_child_samples=31, n_estimators=700, num_leaves=100, random_state=77, subsample=1.0, total= 1.0min
[CV] colsample_bytree=1.0, learning_rate=0.051608013997313074, min_child_samples=48, n_estimators=500, num_leaves=100, random_state=77, subsample=0.7 
[CV]  colsample_bytree=1.0, learning_rate=0.051608013997313074, min_child_samples=48, n_estimators=500, num_leaves=100, random_state=77, subsample=0.7, total=  47.4s
[CV] colsample_bytree=0.9, learning_rate=0.04052426834090161, min_child_samples=42, n_estimators=400, num_leaves=70, random_state=77, subsample=0.9 
[CV]  colsample_bytree=0.9, learning_rate=0.04052426834090161, min_child_samples=42, n_estimators=400, num_leaves=70, random_state=77, subsample=0.9, total=  36.6s
[CV] colsample_bytree=1.0, learning_rate=0.10649753500711753, min_child_samples=17, n_estimators=300, num_leaves=90, random_state=77, subsample=0.9 
[CV]  colsample_bytree=1.0, learning_rate=0.106497535007

[CV]  colsample_bytree=1.0, learning_rate=0.22161815235979518, min_child_samples=13, n_estimators=800, num_leaves=40, random_state=77, subsample=0.9, total=  37.5s
[CV] colsample_bytree=0.9, learning_rate=0.14657524390001295, min_child_samples=14, n_estimators=700, num_leaves=40, random_state=77, subsample=1.0 
[CV]  colsample_bytree=0.9, learning_rate=0.14657524390001295, min_child_samples=14, n_estimators=700, num_leaves=40, random_state=77, subsample=1.0, total=  31.7s
[CV] colsample_bytree=0.8, learning_rate=0.2228867717326785, min_child_samples=10, n_estimators=400, num_leaves=80, random_state=77, subsample=0.8 
[CV]  colsample_bytree=0.8, learning_rate=0.2228867717326785, min_child_samples=10, n_estimators=400, num_leaves=80, random_state=77, subsample=0.8, total=  26.4s
[CV] colsample_bytree=0.8, learning_rate=0.22628362648941788, min_child_samples=36, n_estimators=300, num_leaves=90, random_state=77, subsample=0.9 
[CV]  colsample_bytree=0.8, learning_rate=0.22628362648941788, 

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 85.6min finished


RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0], dtype=int64)),
          error_score='raise',
          estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
       n_jobs=-1, num_leaves=31, objective=None, random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1),
          fit_params=None, iid=True, n_iter=100, n_jobs=1,
          param_distributions={'random_state': [77], 'num_leaves': [40, 50, 60, 70, 80, 90, 100], 'n_estimators': [200, 300, 400, 500, 600, 700, 800], 'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x11d3a4550>, 'min_child_samples': <scipy.stats._distn_infrastructure.rv_frozen object at 0x11af3c828>, 'colsample_bytree': [0.7, 0.8, 0.9, 1.0], 'subsample': [0.7, 0.8, 0.9, 1.0]},

In [93]:
rscv.best_params_

{'colsample_bytree': 0.8,
 'learning_rate': 0.12101431676613092,
 'min_child_samples': 17,
 'n_estimators': 600,
 'num_leaves': 70,
 'random_state': 77,
 'subsample': 1.0}

In [95]:
rscv.best_score_

-0.4918312148437484

In [96]:
# print('train', RMSLE(y_trn, lgb.predict(X_trn)))
# print('valid', RMSLE(y_val, lgb.predict(X_val)))

print('# params')
print(rscv.best_params_)
print()
print('# descriptors')
print(list(X_trn.columns))

# params
{'colsample_bytree': 0.8, 'learning_rate': 0.12101431676613092, 'min_child_samples': 17, 'n_estimators': 600, 'num_leaves': 70, 'random_state': 77, 'subsample': 1.0}

# descriptors
['dow', 'year', 'month', 'day_of_week', 'holiday_flg', 'min_visitors', 'mean_visitors', 'median_visitors', 'max_visitors', 'count_observations', 'latitude', 'longitude', 'rs1_x', 'rv1_x', 'rs2_x', 'rv2_x', 'rs1_y', 'rv1_y', 'rs2_y', 'rv2_y', 'total_reserv_sum', 'total_reserv_mean', 'total_reserv_dt_diff_mean', 'date_int', 'var_max_lat', 'var_max_long', 'lon_plus_lat', 'air_store_id_target_mean', 'air_store_id_target_std', 'air_genre_name_target_mean', 'air_genre_name_target_std', 'air_area_name_target_mean', 'air_area_name_target_std']


In [97]:
test['visitors'] = np.expm1(lgb.predict(X_tst))
filename = '../output/180128_lgb5'
test[['id', 'visitors']].to_csv(filename+'.csv.gz', index=False, compression='gzip')